In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.preprocessing import OrdinalEncoder
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('data/Dataset-SA.csv')
df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [2]:
df['product_name'].iloc[1]

'Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)'

In [3]:
print(df.isnull().sum())
print(df.shape)
print(df['Review'].isnull().sum())
print("null val = {:.2f} %".format((df['Review'].isnull().sum()/df.shape[0])*100))

product_name         0
product_price        0
Rate                 0
Review           24664
Summary             11
Sentiment            0
dtype: int64
(205052, 6)
24664
null val = 12.03 %


In [4]:
df.dropna(inplace=True)

In [5]:
data = df[['Rate','Review','Summary','Sentiment']]

In [6]:
data['Sentiment'].value_counts()

Sentiment
positive    147171
negative     24401
neutral       8807
Name: count, dtype: int64

In [7]:
data = data[data['Sentiment'] !='neutral']

In [8]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re

stpwords = set(stopwords.words('english'))  # Convert stopwords list to a set for fast lookup
stem = PorterStemmer()  # Create an instance

def clean(i):
    i = re.sub(r'READ MORE', '', i)
    i = i.lower()  # Convert to lowercase
    words = word_tokenize(i)  # Tokenize sentence
    words = [stem.stem(word) for word in words if word not in stpwords]
    words = [word for word in words if word not in string.punctuation]
    return ' '.join(words)  # Join words back into a sentence
    


In [9]:
data['Sentiment'].value_counts()

Sentiment
positive    147171
negative     24401
Name: count, dtype: int64

In [10]:
data['Review'] = data['Review'].apply(clean)
data['Summary'] = data['Summary'].apply(clean)
ord = OrdinalEncoder(categories=[['negative','positive']])
data['Sentiment'] = ord.fit_transform(data[['Sentiment']])

In [11]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score

# X = data[['Review','Summary']]
# y = data['Sentiment']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# vectorizer_review = CountVectorizer(ngram_range=(1,2))
# vectorizer_summary = CountVectorizer(ngram_range=(1,2))

# review_bow_train = vectorizer_review.fit_transform(X_train['Review'])
# summary_bow_train = vectorizer_summary.fit_transform(X_train['Summary'])

# review_bow_test = vectorizer_review.transform(X_test['Review'])
# summary_bow_test = vectorizer_summary.transform(X_test['Summary'])

# # Merge Sparse Matrices Horizontally
# from scipy.sparse import hstack

# X_train_bow = hstack([review_bow_train, summary_bow_train])
# X_test_bow = hstack([review_bow_test, summary_bow_test])

# model = MultinomialNB()
# model.fit(X_train_bow, y_train)

# # Predict & Evaluate
# pred = model.predict(X_test_bow)
# print("Accuracy:", accuracy_score(y_test, pred))


In [12]:
# data.to_csv('reviews_filtered.csv',index=False)

In [13]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score

# X = data[['Review','Summary']]
# y = data['Sentiment']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# vectorizer_review = TfidfVectorizer()
# vectorizer_summary = TfidfVectorizer()

# review_bow_train = vectorizer_review.fit_transform(X_train['Review'])
# summary_bow_train = vectorizer_summary.fit_transform(X_train['Summary'])

# review_bow_test = vectorizer_review.transform(X_test['Review'])
# summary_bow_test = vectorizer_summary.transform(X_test['Summary'])

# # Merge Sparse Matrices Horizontally
# from scipy.sparse import hstack

# X_train_bow = hstack([review_bow_train, summary_bow_train])
# X_test_bow = hstack([review_bow_test, summary_bow_test])

# model = MultinomialNB()
# model.fit(X_train_bow, y_train)

# # Predict & Evaluate
# pred = model.predict(X_test_bow)
# print("Accuracy:", accuracy_score(y_test, pred))


In [14]:
# review = "good quality product"
# summary = "cooler is big and wide maharaja brand trust y every thing is good about cooler create less noise only thing is that its motor is not powerful even in high speed rest all things are good but yes for one middle size room it is ok"
# data_sample = [[review,summary]]
# df = pd.DataFrame(data_sample,columns=['Review','Summary'])
# review_bow_test = vectorizer_review.transform(df[['Review']])
# summary_bow_test = vectorizer_summary.transform(df[['Summary']])
# row = hstack([review_bow_test, summary_bow_test])

# print(model.predict(row))


In [15]:
from imblearn.under_sampling import RandomUnderSampler

# Assuming df has 'Review', 'Summary', and 'Sentiment' columns
X = data[['Review', 'Summary']]  # Selecting both text columns
y = data['Sentiment']  # Target column (1 = Positive, 0 = Negative)

# Initialize the undersampler
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Resample the dataset
X_resampled, y_resampled = rus.fit_resample(X, y)

# Create the new balanced DataFrame
df_balanced = pd.DataFrame({'Review': X_resampled['Review'], 
                            'Summary': X_resampled['Summary'], 
                            'Sentiment': y_resampled})

# Display the new balanced dataset
print(df_balanced.head())


                   Review                                            Summary  \
3         useless product                                    bad product fan   
8          unsatisfactori                                         bad cooler   
32  expect better product                                        bad qualiti   
48             wast money  small wire moter capac low fan speed good flow...   
53                   nice  receiv dalay 10 day cooler ok reveiv switch damag   

    Sentiment  
3         0.0  
8         0.0  
32        0.0  
48        0.0  
53        0.0  


In [16]:
df_balanced['Sentiment'].value_counts()

Sentiment
0.0    24401
1.0    24401
Name: count, dtype: int64

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

X = df_balanced[['Review','Summary']]
y = df_balanced['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer_review = CountVectorizer(ngram_range=(1,2))
vectorizer_summary = CountVectorizer(ngram_range=(1,2))

review_bow_train = vectorizer_review.fit_transform(X_train['Review'])
summary_bow_train = vectorizer_summary.fit_transform(X_train['Summary'])

review_bow_test = vectorizer_review.transform(X_test['Review'])
summary_bow_test = vectorizer_summary.transform(X_test['Summary'])

# Merge Sparse Matrices Horizontally
from scipy.sparse import hstack

X_train_bow = hstack([review_bow_train, summary_bow_train])
X_test_bow = hstack([review_bow_test, summary_bow_test])

model = MultinomialNB()
# model.fit(X_train_bow, y_train)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train_bow, y_train, cv=5)    
print("Mean CV Accuracy:", scores.mean())

# Predict & Evaluate
# pred = model.predict(X_test_bow)
# print("Accuracy:", accuracy_score(y_test, pred))
# with open('sentiment_model2.pkl', 'wb') as file:
#     pickle.dump((model, vectorizer_review, vectorizer_summary), file)


Mean CV Accuracy: 0.9369381023682216


In [19]:
# with open('sentiment_model2.pkl', 'rb') as file:
#     model, vectorizer_review, vectorizer_summary = pickle.load(file)

# review = "Perfect product!"
# summary = "Tirggr ultrabuds N1 Neo are awesome sound quality and battery back is awesome simply its abest product in such a nominal amount these buds are very comfortable in wearing its a worth money product highly recommendable."
# data = pd.DataFrame({'Review':[review],'Summary':[summary]})
# review_test = vectorizer_review.transform(data['Review'])
# summary_test = vectorizer_summary.transform(data['Summary'])

# X_test_new = hstack([review_test, summary_test])

# model.predict(X_test_new)


In [ ]:
# with open('sentiment_model.pkl', 'rb') as file:
#     model, vectorizer_review, vectorizer_summary = pickle.load(file)

# data = pd.read_csv('reviews.csv')
# review_test = vectorizer_review.transform(data['Review'])
# summary_test = vectorizer_summary.transform(data['Summary'])

# X_test_new = hstack([review_test, summary_test])

# preds = model.predict(X_test_new)

# print(preds)


In [23]:
test = pd.read_csv('data/reviews_flipkart.csv')
print(test.iloc[0])

Review                                             Brilliant
Summary    A classy earbuds which comes with great sound ...
Name: 0, dtype: object


In [14]:
import re
re.sub(r'READ MORE', '', test.iloc[1]['Summary'])

'Fits perfectly in my ears . Sound is awesome . No lags . Quick charge . Looks good . I use it daily for my work and workouts . I just love them . They are the perfect companions for my daily routine work . Perfectly connects with my phone .'

In [24]:
test['Summary'] = test['Summary'].apply(clean)
test['Review'] = test['Review'].apply(clean)
for i in test['Summary']:
	# i = re.sub(r'READ MORE', '', i)
	print(i)

classi earbud come great sound bass good batteri backup chare good good long time use worth everi penni best budget
fit perfectli ear sound awesom lag quick charg look good use daili work workout love perfect companion daili routin work perfectli connect phone
best connect batteri visual use two year best one prefer other
super qualiti aur super bossso beauti love fantast 😍😍
product good sound also good bass amaz valu money ...
nois cancel good .... extern nois phone call clearli audibl sidd
bud sleek metal design comfort provid realli good music experi signatur sound dual eq provid balanc music experi great game also quad mic enx technolog beast mode 50m low latenc provid great sound qualiti charg pretti fast sweat splash resist huge batteri backup overal valu money
sound qualiti 10/10 .. bass 10/10just go ....
clear sound listen music make call bass decent disappoint batteri backup excel colour design beauti weight normal volum control switch start paus touch button work flawlessli m

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
link = "https://www.flipkart.com/noise-mettle-1-4-display-stainless-steel-finish-metal-strap-bluetooth-calling-smartwatch/p/itm3f95e24755967?pid=SMWGZMH7D5FQFQVV&marketplace=FLIPKART"
review_link = re.sub(r'/p/','/product-reviews/',link)
review = []
summary = []
c = 1
while c<4:
	print(review_link)
	soup = BeautifulSoup(requests.get(review_link).content, 'html.parser')
	print('page')
	review_block = soup.find_all('div',"col EPCmJX Ma1fCG")
	for i in review_block:
		rvsoup = BeautifulSoup(str(i), 'html.parser')
		review.append(rvsoup.find('p','z9E0IG').text)
		summary.append(rvsoup.find('div','ZmyHeo').text)
	next_link = soup.find_all('a','_9QVEpD')[-1].get('href')
	
	review_link = 'https://www.flipkart.com'+next_link
	c +=1
data = pd.DataFrame({'Review':review,'Summary':summary})
# for i in data['Summary']:
# 	print(i)
# return data


https://www.flipkart.com/noise-mettle-1-4-display-stainless-steel-finish-metal-strap-bluetooth-calling-smartwatch/product-reviews/itm3f95e24755967?pid=SMWGZMH7D5FQFQVV&marketplace=FLIPKART
page
https://www.flipkart.com/noise-mettle-1-4-display-stainless-steel-finish-metal-strap-bluetooth-calling-smartwatch/product-reviews/itm3f95e24755967?pid=SMWGZMH7D5FQFQVV&marketplace=FLIPKART&page=2
page
https://www.flipkart.com/noise-mettle-1-4-display-stainless-steel-finish-metal-strap-bluetooth-calling-smartwatch/product-reviews/itm3f95e24755967?pid=SMWGZMH7D5FQFQVV&marketplace=FLIPKART&page=1
page


In [17]:
link = "https://www.flipkart.com/noise-mettle-1-4-display-stainless-steel-finish-metal-strap-bluetooth-calling-smartwatch/product-reviews/itm3f95e24755967?pid=SMWGZMH7D5FQFQVV&lid=LSTSMWGZMH7D5FQFQVVFUZN9N&marketplace=FLIPKART&page=2"
soup = BeautifulSoup(requests.get(link).content, 'html.parser')
linkn = soup.find_all('a','_9QVEpD')[-1].get('href')
print(linkn)


/noise-mettle-1-4-display-stainless-steel-finish-metal-strap-bluetooth-calling-smartwatch/product-reviews/itm3f95e24755967?pid=SMWGZMH7D5FQFQVV&lid=LSTSMWGZMH7D5FQFQVVFUZN9N&marketplace=FLIPKART&page=3


In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

df_balanced.head()

,Review,Summary,Sentiment
3,useless product,bad product fan,0.0
8,unsatisfactori,bad cooler,0.0
32,expect better product,bad qualiti,0.0
48,wast money,small wire moter capac low fan speed good flow...,0.0
53,nice,receiv dalay 10 day cooler ok reveiv switch damag,0.0


In [19]:
df_balanced["Combine"] = df_balanced['Review'] + df_balanced['Summary']
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")  # Keep top 10,000 words
tokenizer.fit_on_texts(df_balanced["Combine"])
sequences = tokenizer.texts_to_sequences(df_balanced["Combine"])
padded_sequences = pad_sequences(sequences, maxlen=100, padding="post", truncating="post")


In [20]:
X_train,X_test,y_train,y_test = train_test_split(padded_sequences, df_balanced['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
model = Sequential()

# model.add(Embedding(10000, output_dim=64,input_length=100))
model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64))
# model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)